In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

try:
    df = pd.read_csv("../project1/attacks.csv", encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv("../project1/attacks.csv", encoding='latin1') 

df.columns = df.columns.str.lower().str.replace(' ', '_')

In [2]:
# Creating a subset with the info I need
subset = df[["date", "year", "type", "country", "area", "location", "activity", "name", "sex_","fatal_(y/n)", "species_", "investigator_or_source"]]

In [3]:
#Dropping the rows that are all in blank
subset = subset.dropna(how='all')

In [4]:
# Cleaning column 'date' and Creating df = date with only the info that match the regex pattern

pattern = r'\d{2}-[A-Za-z]{3}-\d{4}$'

# Using the pattern to create a subset with only the data that match: DD-Mon-YYYY
subset = subset[subset["date"].str.match(pattern, 
                                       na=False, 
                                       case=False)]

# Formating column 'Date'into datetime
subset["date"] = pd.to_datetime(subset.date,format="mixed")

# Formating column 'Year' into float
subset["year"] = subset["year"].astype(float)

# Adding a column named "Weekday"
subset["weekday"] = subset["date"].dt.weekday

In [5]:
# Cleaning the column 'fatal_(y|n)' 

def clean_fatal(fatal):
    if isinstance(fatal, str):
        # Checking if has Y or N - can be with space before or after...
        match = re.match(r'\s*(Y|N)\s*', fatal, re.I)  # the re.I means will get upper and lower case
        if match:
            return match.group(1).upper()  # return Y or N in uppercase
    return 'Unknown'

# Apply the clean_fatal function to the 'Fatal (Y/N)' column
subset['fatal_clean'] = subset['fatal_(y/n)'].apply(clean_fatal)

NameError: name 're' is not defined

In [ ]:
# Cleaning the column 'specie_' and Creating a new column named 'species_clean'

species_mapping = {
    'white shark': 'White shark',
    'tiger shark': 'Tiger shark',
    'bull shark': 'Bull shark',
}

# Function to check the species that was given in the data set
def clean_species(species):
    if isinstance(species, str): # Only do lower case in the one are not blank
        species = species.lower() 
        for key, value in species_mapping.items():
            if key in species:
                # The re.escape() will espace the special characters
                pattern = re.escape(key) + r'.*' # Get everythin that match the species mapping
                if re.search(pattern, species): 
                    return value # If True return the Value Y or N
    return 'Unknown' # if pattern is not in species will return ''

subset["species_clean"] = subset["species_"].apply(clean_species)


In [ ]:
subset.columns

Index(['date', 'year', 'type', 'country', 'area', 'location', 'activity',
       'name', 'sex_', 'fatal_(y/n)', 'species_', 'investigator_or_source',
       'weekday', 'fatal_clean', 'species_clean'],
      dtype='object')

In [ ]:
# Exporting the data frame that was created in new csv directory

subset.to_csv('../project1/newdf.csv')